<a href="https://colab.research.google.com/github/RonitGandhi/Hacksprint_PS01_InputOutput/blob/model_branch/SENTIMETER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import os
import re

import nltk
import pandas as pd

nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
currency = [
    "DOGE"
]

In [ ]:
def process_ccy(curr):
    pd_df = pd.concat(
        [pd.read_csv(f) for f in glob.glob(f"clean_data/{curr}*.csv", recursive=True)],
        ignore_index=True,
    )

    print(f"Number of records loaded for {curr}", len(pd_df))
    #pd_df[["coin_symbol", "tweet_id", "created_at", "date", "hour"]].to_csv(
    #    f"clean_data/ccy_tweets/{curr}_tweets.csv", index=False
    #)

    return pd_df

In [ ]:
combined_df = pd.concat(
    [process_ccy(c) for c in currency],
    ignore_index=True,
)

Number of records loaded for DOGE 17043


In [ ]:
combined_df.sort_values(by=["mined_at"], inplace=True, ignore_index=True)
combined_df.drop_duplicates(
    subset=["tweet_id"], inplace=True, keep="last", ignore_index=True
)
combined_df.head(5)

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,followers_count,location,source_device,retweet_text
0,1446836027986006016,savas,savasuctu,218.0,RT @RealFlokiInu: Please verify our account @v...,2021-10-09 13:52:46.043200,2021-10-09 13:52:44.000000,0.0,"[{'text': 'FLOKI', 'indices': [78, 84]}, {'tex...",2722,93.0,NaN,Twitter Web App,Please verify our account @verified &amp; @Twi...
1,1446836023615365133,😇 Adam H0K1 BigWin 😇 aminn,SiChubby_H0k1,5070.0,RT @RockstarDogeBSC: ✅ WE ARE NOW OFFICIALLY L...,2021-10-09 13:52:46.057235,2021-10-09 13:52:43.000000,0.0,"[{'text': 'ROCKSTARDOGE', 'indices': [76, 89]}...",274,10.0,NaN,Twitter for Android,✅ WE ARE NOW OFFICIALLY LISTED ON COINMARKETCA...
2,1446836020645933063,Bogdan V,bogdan_vatavu,218.0,RT @RealFlokiInu: Please verify our account @v...,2021-10-09 13:52:46.068569,2021-10-09 13:52:42.000000,0.0,"[{'text': 'FLOKI', 'indices': [78, 84]}, {'tex...",1897,30.0,NaN,Twitter for iPhone,Please verify our account @verified &amp; @Twi...
3,1446836009585373197,CHARLIΞ👺,CryptoCharlieva,196.0,RT @Dra_Claudiaa: I’m #doge hodler! #hodl #dog...,2021-10-09 13:52:46.077750,2021-10-09 13:52:40.000000,0.0,"[{'text': 'doge', 'indices': [22, 27]}, {'text...",570,193.0,NaN,Twitter for iPhone,I’m #doge hodler! #hodl #dogecoin #shibainu 💙💙...
4,1446836008738168832,MONEY-BOY X,MB_3213,0.0,Same thing to #dogefather #dogearmy #DogecoinT...,2021-10-09 13:52:46.086650,2021-10-09 13:52:40.000000,0.0,"[{'text': 'dogefather', 'indices': [14, 25]}, ...",17090,569.0,NaN,Twitter for iPhone,None


In [ ]:
#combined_df.sort_values(by=["tweet_id"], inplace=True)

In [ ]:
len(combined_df)

17042

In [ ]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

tokenizer = RegexpTokenizer(r"\w+")

In [ ]:
def preprocess(text):
    words = [lemmatizer.lemmatize(w) for w in text if w not in stop_words]
    stem_text = " ".join([stemmer.stem(i) for i in words])
    return stem_text

In [ ]:
combined_df = combined_df.astype({"text": 'str'})
combined_df["processed text"] = combined_df["text"].apply(
    lambda x: preprocess(tokenizer.tokenize(x.lower()))
)

In [ ]:
def getSentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

In [ ]:
combined_df["polarity"] = combined_df["processed text"].apply(lambda x: getSentiment(x))

In [ ]:
combined_df.head(5)

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,followers_count,location,source_device,retweet_text,processed text,polarity
0,1446836027986006016,savas,savasuctu,218.0,RT @RealFlokiInu: Please verify our account @v...,2021-10-09 13:52:46.043200,2021-10-09 13:52:44.000000,0.0,"[{'text': 'FLOKI', 'indices': [78, 84]}, {'tex...",2722,93.0,NaN,Twitter Web App,Please verify our account @verified &amp; @Twi...,rt realflokiinu pleas verifi account verifi am...,0.0
1,1446836023615365133,😇 Adam H0K1 BigWin 😇 aminn,SiChubby_H0k1,5070.0,RT @RockstarDogeBSC: ✅ WE ARE NOW OFFICIALLY L...,2021-10-09 13:52:46.057235,2021-10-09 13:52:43.000000,0.0,"[{'text': 'ROCKSTARDOGE', 'indices': [76, 89]}...",274,10.0,NaN,Twitter for Android,✅ WE ARE NOW OFFICIALLY LISTED ON COINMARKETCA...,rt rockstardogebsc offici list coinmarketcap r...,0.0
2,1446836020645933063,Bogdan V,bogdan_vatavu,218.0,RT @RealFlokiInu: Please verify our account @v...,2021-10-09 13:52:46.068569,2021-10-09 13:52:42.000000,0.0,"[{'text': 'FLOKI', 'indices': [78, 84]}, {'tex...",1897,30.0,NaN,Twitter for iPhone,Please verify our account @verified &amp; @Twi...,rt realflokiinu pleas verifi account verifi am...,0.0
3,1446836009585373197,CHARLIΞ👺,CryptoCharlieva,196.0,RT @Dra_Claudiaa: I’m #doge hodler! #hodl #dog...,2021-10-09 13:52:46.077750,2021-10-09 13:52:40.000000,0.0,"[{'text': 'doge', 'indices': [22, 27]}, {'text...",570,193.0,NaN,Twitter for iPhone,I’m #doge hodler! #hodl #dogecoin #shibainu 💙💙...,rt dra_claudiaa doge hodler hodl dogecoin shib...,0.0
4,1446836008738168832,MONEY-BOY X,MB_3213,0.0,Same thing to #dogefather #dogearmy #DogecoinT...,2021-10-09 13:52:46.086650,2021-10-09 13:52:40.000000,0.0,"[{'text': 'dogefather', 'indices': [14, 25]}, ...",17090,569.0,NaN,Twitter for iPhone,None,thing dogefath dogearmi dogecointothemoon doge...,0.0


In [ ]:
combined_df["sentiment"] = combined_df["polarity"].apply(
    lambda s: "Postive" if s > 0 else ("Neutral" if s == 0 else "Negative")
)

In [ ]:
combined_df.to_csv("DOGE_sentiment.csv", index=False)

In [ ]:
combined_df['sentiment'].value_counts()

Neutral     9269
Postive     6602
Negative    1171
Name: sentiment, dtype: int64

In [ ]:
eth_df = combined_df